In [153]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [155]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [156]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [157]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [158]:
df_market_data.columns

Index(['price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d',
       'price_change_percentage_60d', 'price_change_percentage_200d',
       'price_change_percentage_1y'],
      dtype='object')

In [159]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_df_scaled = StandardScaler().fit_transform(df_market_data[['price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d', 'price_change_percentage_60d']])
# Display the first 5 rows of the scaled data
market_df_scaled[0:5]

array([[ 0.50852937,  0.49319307,  0.77220043,  0.23545963, -0.0674951 ],
       [ 0.18544589,  0.93444504,  0.55869212, -0.05434093, -0.27348273],
       [ 0.02177396, -0.70633685, -0.02168042, -0.06103015,  0.00800452],
       [-0.04076438, -0.81092807,  0.24945797, -0.05038797, -0.37316402],
       [ 1.19303608,  2.00095907,  1.76061001,  0.54584206, -0.29120287]])

In [160]:
# Create a DataFrame with the scaled data
market_df_scaled = pd.DataFrame(market_df_scaled, columns=['price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d', 'price_change_percentage_60d'])

# Copy the crypto names from the original DataFrame
market_df_scaled["coin_id"] = list(df_market_data.index)

# Set the coin_id column as index
market_df_scaled = market_df_scaled.set_index("coin_id")

# Display the scaled DataFrame
market_df_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d
coin_id,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203


---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [161]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,11))

In [162]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    model = KMeans(n_clusters=i)
    model.fit(market_df_scaled)
    inertia.append(model.inertia_)

In [163]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    "k": k,
    "inertia": inertia
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

In [198]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
elbow_1_plot = df_elbow.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=k
)
elbow_1_plot

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer: 4

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [186]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4)

In [187]:
# Fit the K-Means model using the scaled DataFrame
model.fit(market_df_scaled)

KMeans(n_clusters=4)

In [188]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
predictions = model.predict(market_df_scaled)

# Print the resulting array of cluster values.
print(predictions)

[0 0 2 2 0 0 0 0 0 2 2 2 2 0 2 0 2 2 0 2 2 0 2 2 2 2 2 2 0 2 2 2 3 0 2 2 1
 2 2 2 2]


In [189]:
# Create a copy of the scaled DataFrame
market_df_predictions = market_df_scaled.copy()

In [190]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
market_df_predictions["clusters"] = predictions

# Display the copy of the scaled DataFrame
market_df_predictions.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,clusters
coin_id,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,0
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,0
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,2
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,2
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,0


In [202]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
scatter_plot_1 = market_df_predictions.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="clusters",
    hover_cols=["coin_id"],
    title="Cryptocurrency Clusters"
)
scatter_plot_1

:NdOverlay   [clusters]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [171]:
# Create a PCA model instance and set `n_components=3`.
pca=PCA(n_components=3)

In [172]:
# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
crypto_pca = pca.fit_transform(market_df_scaled)

# View the scaled PCA data
crypto_pca[0:5]

array([[ 0.79510406,  0.66290603,  0.15388459],
       [ 0.41376913,  1.04777288, -0.15762251],
       [-0.19550822, -0.51826138,  0.20153516],
       [-0.26074757, -0.34060069,  0.14574095],
       [ 1.96101883,  2.23972436,  0.18298129]])

In [173]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
pca.explained_variance_ratio_

array([0.47862164, 0.26608254, 0.1684978 ])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 

In [ ]:
# Create a new DataFrame with the PCA data.
crypto_pca_df = pd.DataFrame(
    data=crypto_pca,
    columns=["PC 1", "PC 2", "PC 3"],
    index=market_df_scaled.index
)


# Display the scaled PCA DataFrame
crypto_pca_df.head()

,PC 1,PC 2,PC 3
coin_id,,,
bitcoin,0.795104,0.662906,0.153885
ethereum,0.413769,1.047773,-0.157623
tether,-0.195508,-0.518261,0.201535
ripple,-0.260748,-0.340601,0.145741
bitcoin-cash,1.961019,2.239724,0.182981


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [191]:
# Create a list with the number of k-values from 1 to 11
k_pca = list(range(1,11))

In [193]:
# Create an empty list to store the inertia values
inertia_pca = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k_pca:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(crypto_pca_df)
    inertia_pca.append(model.inertia_)

In [194]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {"k": k_pca, "inertia": inertia_pca}
# Create a DataFrame with the data to plot the Elbow curve
elbow_df_pca = pd.DataFrame(elbow_data_pca)

In [199]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
elbow_pca_plot = elbow_df_pca.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve for PCA Data",
    xticks=k_pca
)
elbow_pca_plot

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer: 5


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer: Only a slight difference. 

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [179]:
# Initialize the K-Means model using the best value for k
model2 = KMeans(n_clusters=5, random_state=0)

In [180]:
# Fit the K-Means model using the PCA data
model2.fit(crypto_pca_df)

KMeans(n_clusters=5, random_state=0)

In [181]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame

predictiions2 = model2.predict(crypto_pca_df)
# Print the resulting array of cluster values.
print(predictiions2)

[1 1 0 0 3 3 3 3 1 0 0 0 0 3 1 1 0 0 3 0 0 1 0 0 0 0 1 0 1 0 1 0 4 1 0 0 2
 0 1 0 0]


In [182]:
# Create a copy of the scaled PCA DataFrame
crypto_pca_df_predictions = crypto_pca_df.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
crypto_pca_df_predictions["pca_clusters"] = predictiions2

# Display the copy of the scaled PCA DataFrame
crypto_pca_df_predictions.head()

,PC 1,PC 2,PC 3,pca_clusters
coin_id,,,,
bitcoin,0.795104,0.662906,0.153885,1
ethereum,0.413769,1.047773,-0.157623,1
tether,-0.195508,-0.518261,0.201535,0
ripple,-0.260748,-0.340601,0.145741,0
bitcoin-cash,1.961019,2.239724,0.182981,3


In [201]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
pca_scatter_plot = crypto_pca_df_predictions.hvplot.scatter(
    x="PC 1",
    y="PC 2",
    by="pca_clusters",
    hover_cols=["coin_id"],
    title="Cryptocurrency Clusters with PCA Data"
)
pca_scatter_plot

:NdOverlay   [pca_clusters]
   :Scatter   [PC 1]   (PC 2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [205]:
# Composite plot to contrast the Elbow curves
elbow_1_plot + elbow_pca_plot


:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [203]:
# Composite plot to contrast the clusters
scatter_plot_1 + pca_scatter_plot

:Layout
   .NdOverlay.I  :NdOverlay   [clusters]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [pca_clusters]
      :Scatter   [PC 1]   (PC 2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer: Applying Principal Component Analysis (PCA) to reduce the number of features before running the KMeans allows for more clearly defined and separated clusters. PCA accomplishes this by filtering out the noise and keeping the most variance in the data. Clearer clusters and separation help us better interpret the data. 